In [71]:
import os
import json
import pandas as pd

PATH = 'data/edgar_results/'

In [81]:
dfs = []
for file in os.listdir(PATH):
    if not file.endswith('.json'):
        continue
    data = json.loads(open(PATH + file).read())
    if len(data['entries']) > 0:
        non_zero_entries.append(data)
        df = pd.DataFrame.from_dict(data['entries'])[['link', 'filing-date', 'size']]
        df['filing_date'] = pd.to_datetime(df['filing-date'])
        df['year'] = df.filing_date.dt.year
        df['cik'] = data['feed']['cik']
        df['conformed_name'] = data['feed']['conformed-name']
        df = df.drop(['filing-date'], axis=1)
        dfs.append(df)
df = pd.concat(dfs).reset_index(drop=True)

In [82]:
df.tail()

,link,size,filing_date,year,cik,conformed_name
6725,https://www.sec.gov/Archives/edgar/data/135544...,59 KB,2018-05-29,2018,0001355444,EMBRAER S.A.
6726,https://www.sec.gov/Archives/edgar/data/135544...,85 KB,2017-05-16,2017,0001355444,EMBRAER S.A.
6727,https://www.sec.gov/Archives/edgar/data/135544...,658 KB,2016-05-24,2016,0001355444,EMBRAER S.A.
6728,https://www.sec.gov/Archives/edgar/data/135544...,64 KB,2015-05-29,2015,0001355444,EMBRAER S.A.
6729,https://www.sec.gov/Archives/edgar/data/135544...,60 KB,2014-05-30,2014,0001355444,EMBRAER S.A.


In [83]:
df.year.unique()

array([2019, 2018, 2017, 2016, 2015, 2014])

In [84]:
for year in df.year.unique():
    df[df.year == year].to_csv(f'summary_data/sd-filing-info-{year}.csv')